### Extract vocabulary

In [25]:
from pathlib import Path

DATADIR = "../data/ner_on_html"

def get_sentences(filename):    
    with Path(DATADIR, filename).open('r', encoding="utf-8") as f:
        sentences = f.read().strip().split('\n\n')
        return [[t.split() for t in s.split('\n')] for s in sentences if len(s) > 0] 

def extract_vocabulary(filenames):
    if not isinstance(filenames, list):
        filenames = [filenames]
     
    words = []
    for f in filenames:
        sentences = get_sentences(f)
        # words = words + [w[14] if len(w) > 14 else 'none' for s in get_sentences(f) for w in s if not w[0] == '-DOCSTART-']
        words = words + [w[0] for s in get_sentences(f) for w in s if not w[0] == '-DOCSTART-']
    
    words = list(set(words))

    with Path(DATADIR, 'vocab.words.txt').open('w', encoding='utf8') as f:
        for w in words:
            f.write(w + '\n')

    print('Found %d words.' % (len(words)))            
    
extract_vocabulary(['train', 'valid', 'test'])

Found 35206 words.


In [ ]:
# -*- coding: utf-8 -*-

LOWERCASE = False

from pathlib import Path

import numpy as np

if __name__ == '__main__':
  DATADIR = "../data/ner_on_html"
    
  # Load vocab.
  with Path(DATADIR, 'vocab.words.txt').open(encoding='utf-8') as f:
    word_to_idx = {line.strip(): idx for idx, line in enumerate(f)}
  size_vocab = len(word_to_idx)

  word_to_lowercase = {}
  for key in word_to_idx:
    lkey = key.lower()
    if not lkey in word_to_lowercase:
      word_to_lowercase[lkey] = []
    word_to_lowercase[lkey].append(word_to_idx[key])
    
  # Array of zeros
  embeddings = np.zeros((size_vocab, 300))

  # Get relevant glove vectors
  found = 0
  print('Reading GloVe file (may take a while)')
  with open('../data/glove.840B.300d.txt', encoding="utf-8") as f:
    for line_idx, line in enumerate(f):
      if line_idx % 100000 == 0:
        print('- At line {}'.format(line_idx))
      line = line.strip().split()
      if len(line) != 300 + 1:
        continue
      word = line[0]
      embedding = line[1:]
        
      if LOWERCASE:
        word = word.lower()
        if word in word_to_lowercase:
          for word_idx in word_to_lowercase[word]:
            found += 1
            embeddings[word_idx] = embedding
      else:  
        if word in word_to_idx:
          found += 1
          word_idx = word_to_idx[word]
          embeddings[word_idx] = embedding
  print('- done. Found {} vectors for {} words'.format(found, size_vocab))

  # ASHES = embeddings[22491]
  # ashes = embeddings[2500]
  # print(ASHES)
  # print(ashes)  

  # Save np.array to file
  np.savez_compressed(DATADIR + '/glove.npz', embeddings=embeddings)

Reading GloVe file (may take a while)
- At line 0
- At line 100000
- At line 200000
- At line 300000
- At line 400000
- At line 500000
- At line 600000
- At line 700000
- At line 800000
- At line 900000
- At line 1000000
- At line 1100000
- At line 1200000
- At line 1300000
- At line 1400000
- At line 1500000
- At line 1600000
- At line 1700000
- At line 1800000
- At line 1900000
- At line 2000000
- At line 2100000
- done. Found 23614 vectors for 35214 words
[  2.59580000e-01   2.16440000e-01   1.30040000e-01  -2.94070000e-01
   8.21910000e-01   1.36230000e-01  -9.26550000e-01   2.29270000e-01
   2.97440000e-02  -1.21430000e+00   5.60850000e-03   4.58670000e-01
   2.47650000e-02  -4.82460000e-01  -2.17760000e-01   2.11160000e-01
  -3.97360000e-01  -1.62720000e-01   6.07000000e-01  -8.15420000e-01
   1.94690000e-02  -1.65000000e-01   1.47050000e-01  -4.87490000e-01
  -2.32400000e-01   9.10180000e-02   1.37340000e-01  -1.03210000e-01
   7.15740000e-02   1.59020000e-01   2.16280000e-02  -

In [38]:
# -*- coding: utf-8 -*-

LOWERCASE = False

from pathlib import Path
import re
import numpy as np

if __name__ == '__main__':
  DATADIR = "data"
    
  # Load vocab.
  with Path(DATADIR, 'conll2003/vocab.chars.txt').open(encoding='utf-8') as f:
    char_to_idx = {line.strip(): idx for idx, line in enumerate(f)}
  size_vocab = len(char_to_idx)
  print(size_vocab)
    
  # Array of zeros
  embeddings = np.zeros((size_vocab, 300))
  # Get relevant glove vectors
  found = 0
  print('Reading GloVe file (may take a while)')
  with open('data/char_embeddings.txt', encoding="utf-8") as f:
    for line_idx, line in enumerate(f):
      line = line.strip().split()
      if len(line) != 300 + 1:
        continue  
    
      char = line[0]
      embedding = line[1:]

      if char in char_to_idx:
        found += 1
        char_idx = char_to_idx[char]
        embeddings[char_idx] = embedding
  print('- done. Found {} vectors for {} words'.format(found, size_vocab))

  # Save np.array to file
  np.savez_compressed(DATADIR + '/char_embeddings.npz', embeddings=embeddings)

89
Reading GloVe file (may take a while)
- done. Found 89 vectors for 89 words


In [75]:
import tensorflow as tf

def create_position_embeddings(self, max_length, emb_dim):
    
max_length=1600
emb_dim=    
position_emb = np.array([
    [(pos+1) / np.power(10000, 2 * (j // 2) / emb_dim) for j in range(emb_dim)]
    for pos in range(max_length)
])

position_emb[:,0::2] = np.sin(position_emb[:,0::2]) # dim 2i
position_emb[:,1::2] = np.cos(position_emb[:,1::2]) # dim 2i+1
position_emb = np.vstack([position_emb, [[0.] * emb_dim]])

print(position_emb)
 
    # variable = np.vstack([position_emb, [[0.] * emb_dim]])
    # variable = tf.Variable(variable, dtype=tf.float32, trainable=False)
 
    # seq = tf.constant(np.arange(1600), dtype=tf.int32)
    # seq = tf.nn.embedding_lookup(variable, seq)
    # pos_embeddings = tf.slice(seq, [0], [tf.shape(inputs)[1]])
    # return inputs + pos_embeddings

[[  8.41470985e-01   5.40302306e-01   5.15138753e-01 ...,   9.99999942e-01
    1.84784979e-04   9.99999983e-01]
 [  9.09297427e-01  -4.16146837e-01   8.83057855e-01 ...,   9.99999767e-01
    3.69569951e-04   9.99999932e-01]
 [  1.41120008e-01  -9.89992497e-01   9.98611031e-01 ...,   9.99999475e-01
    5.54354911e-04   9.99999846e-01]
 ..., 
 [ -7.68254661e-01  -6.40144339e-01   7.46905556e-01 ...,   9.99865690e-01
    8.86956273e-03   9.99960665e-01]
 [ -9.53752653e-01   3.00592544e-01   9.82709113e-01 ...,   9.99860035e-01
    9.05434029e-03   9.99959009e-01]
 [ -2.62374854e-01   9.64966028e-01   9.37667769e-01 ...,   9.99854264e-01
    9.23911754e-03   9.99957318e-01]]


array([[  8.41470985e-01,   5.40302306e-01,   5.15138753e-01, ...,
          9.99999942e-01,   1.84784979e-04,   9.99999983e-01],
       [  9.09297427e-01,  -4.16146837e-01,   8.83057855e-01, ...,
          9.99999767e-01,   3.69569951e-04,   9.99999932e-01],
       [  1.41120008e-01,  -9.89992497e-01,   9.98611031e-01, ...,
          9.99999475e-01,   5.54354911e-04,   9.99999846e-01],
       ..., 
       [ -7.68254661e-01,  -6.40144339e-01,   7.46905556e-01, ...,
          9.99865690e-01,   8.86956273e-03,   9.99960665e-01],
       [ -9.53752653e-01,   3.00592544e-01,   9.82709113e-01, ...,
          9.99860035e-01,   9.05434029e-03,   9.99959009e-01],
       [ -2.62374854e-01,   9.64966028e-01,   9.37667769e-01, ...,
          9.99854264e-01,   9.23911754e-03,   9.99957318e-01]])

In [76]:
import tensorflow as tf
import numpy as np
from pathlib import Path
from model.data_loader import DL
import tensorflow_hub as hub
import time

start_time = time.time()

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)

embs= np.zeros((302812, 1024))

with tf.Session() as sess:
  sess.run([tf.initializers.global_variables(), tf.tables_initializer()])

  for f in ['train']:
    print(f)
    iterator = DL().input_fn(f, training=train).make_initializable_iterator()
    next_el = iterator.get_next()
    sess.run(iterator.initializer)

    sentences = []
    n_words = []
    uids_ = []
    
    z = 0
    u_ = 0
    while True:
      try:
        features, labels = sess.run(next_el)
        (words, uids, nwords), (_, _) = features

        words = [w.decode("utf-8") for w in words[0]]
        sentences.append(words)
        n_words.append(nwords[0])
        uids_.append(uids[0])
        
        z += 1 
        print(z) 
        if z >= 500:
          break
      except tf.errors.OutOfRangeError: 
        break
    
    max_len = max([len(s) for s in sentences])
    for i, s in enumerate(sentences):
        sentences[i] = s + [''] * (max_len - len(s))

    embeddings = elmo(
      inputs={
        "tokens": sentences,
        "sequence_len": n_words 
      },
      signature="tokens",
      as_dict=True
    )["elmo"]

    e = sess.run(embeddings)

    for i, _ in enumerate(e):
      for j in range(n_words[i]): 
        uid = uids_[i][j]       
        embs[uid,:] = e[i,j:]
        
    print('Elapsed time: %.4f' % (time.time() - start_time))

  #np.savez_compressed('data/elmo_embeddings.npz', embeddings=embs)


INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3265, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-76-5580cd1cc817>", line 10, in <module>
    elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow_hub/module.py", line 144, in __init__
    self._spec = as_module_spec(spec)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow_hub/module.py", line 33, in as_module_spec
    return load_module_spec(spec)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow_hub/module.py", line 57, in load_module_spec
    path = registry.resolver(path)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow_hub/registry.py", line 42, in __call__
    return impl(*args, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow_hub/compressed_module_resolver.py", line 102, in __call__
    self._lock_f

KeyboardInterrupt: 